In [7]:
import tensorflow as tf
import numpy as np

In [8]:
tf.__version__

'2.6.0'

In [ ]:
from scipy import misc
import os

 # ----------------------------
 # Parsing function with standard python:

 def zip_data_parser(zip_fname):
     os.system('unzip {0}'.format(zip_fname)) # unzip
     folder_name = zip_fname.rsplit('.zip')[0]

     # load data:
     x_stack = []
     y_stack = []
     for i in range(n_images):
         x_stack.append(misc.imread(folder_name + '/image-{0}.png'.format(i)))
         y_stack.append(misc.imread(folder_name + '/mask-{0}.png'.format(i)))
     x = np.array(x_stack)
     y = np.array(y_stack)

     os.system('rm -rf {0}'.format(folder_name)) # remove unzipped folder
     return x, y 

 # ----------------------------
 # Dataset pipeline:

 all_zip_paths = ['../data/'] # list of paths for each zip file
 train_data = tf.constant(all_zip_paths)
 train_data = tf.data.Dataset.from_tensor_slices(train_data)

 train_data = train_data.map(
            lambda filename: tf.py_func(  # Parse the record into tensors
                zip_data_parser,
                [filename],
                [tf.float32, tf.float32]), num_parallel_calls=num_threads)

 # un-batch first, then batch the data again to have dimension [batch_size, N, M, C]
 train_data = train_data.apply(tf.data.experimental.unbatch())
 train_data = train_data.batch(b_size, drop_remainder=True)

In [12]:
print('Loading data...\n')
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print('MNIST dataset loaded.\n')

Loading data...

MNIST dataset loaded.



In [13]:
x_train = x_train/255.0

In [16]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [17]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
print('Training model...\n')
model.fit(x_train, y_train, epochs=3, batch_size=32)

print('Model trained successfully!')

Training model...

Epoch 1/3
1875/1875 [==============================] - 8s 3ms/step - loss: 0.4426 - accuracy: 0.8755
Epoch 2/3
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2481 - accuracy: 0.9295
Epoch 3/3
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2100 - accuracy: 0.9396
Model trained successfully!
